Install dependencies and import libraries.

In [35]:
%%capture

!pip install langgraph
!pip install langgraph-sdk
!pip install langgraph-checkpoint-sqlite
!pip install langchain-community
!pip install langchain-core
!pip install langchain-openai
!pip install chromadb

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI


import chromadb

from pydantic import BaseModel
from typing import Optional, Callable

import uuid
import json
import os
import types
import inspect
import sys

**Clients**


Import API keys and instantiate clients.

In [ ]:
os.environ['OPENAI_API_KEY'] = 'YOUR-API-KEY'
llm = ChatOpenAI(model='gpt-4o-mini')

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name='bug-reports')

**Define Agent state**


We'll define the state that our agent will maintain throughout its operation.

In [ ]:
class State(BaseModel):
    function: Callable
    function_string: str
    arguments: list
    error: bool
    error_description: str = ''
    new_function_string: str = ''
    bug_report: str = ''
    memory_search_results: list = []
    memory_ids_to_update: list = []

**Define Code Healing Node Functions**

Now we'll define the code healing node functions that our agent will use: code_execution_node, code_update_node and code_patching_node.

In [ ]:
def code_execution_node(state: State):
    ''' Run Arbitrary Code '''
    try:
        print('\nRunning Arbitrary Function')
        print('--------------------------\n')
        result = state.function(*state.arguments)
        print('\n✅ Arbitrary Function Ran Without Error')
        print(f'Result: {result}')
        print('---------------------------------------\n')
    except Exception as e:
        print(f'❌ Function Raised an Error: {e}')
        state.error = True
        state.error_description = str(e)
    return state


def code_update_node(state: State):
    ''' Update Arbitratry Code '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with fixing a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}'
        'You must provide a fix for the present error only.'
        'The bug fix should handle the thrown error case gracefully by returning an error message.'
        'Do not raise an error in your bug fix.'
        'The function must use the exact same name and parameters.'
        'Your response must contain only the function definition with no additional text.'
        'Your response must not contain any additional formatting, such as code delimiters or language declarations.'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    new_function_string = llm.invoke([message]).content.strip()

    print('\n🐛 Buggy Function')
    print('-----------------\n')
    print(state.function_string)
    print('\n🩹 Proposed Bug Fix')
    print('-------------------\n')
    print(new_function_string)

    state.new_function_string = new_function_string
    return state


def code_patching_node(state: State):
    ''' Fix Arbitrary Code '''
    try:
        print('\n*******************')
        print('\n❤️‍🩹 Patching code...')
        # Store the new function as a string
        new_code = state.new_function_string

        # Create namespace for new function
        namespace = {}

        # Execute new code in namespace
        exec(new_code, namespace)

        # Get function name dynamically
        func_name = state.function.__name__

        # Get the new function using dynamic name
        new_function = namespace[func_name]

        # Update state
        state.function = new_function
        state.error = False

        # Test the new function
        result = state.function(*state.arguments)

        print('...patch complete 😬\n')

    except Exception as e:
        print(f'...patch failed: {e}')
        print(f'Error details: {str(e)}')

    print('******************\n')
    return state

In [ ]:
def bug_report_node(state: State):
    ''' Generate Bug Report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with generating a bug report for a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}'
        'Your response must be a comprehensive string including only crucial information on the bug report'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    bug_report = llm.invoke([message]).content.strip()

    print('\n📝 Generating Bug Report')
    print('------------------------\n')
    print(bug_report)

    state.bug_report = bug_report
    return state


# Digest the bug report using the same template used when saving bug reports to increase the accuracy and relevance of results when querying the vector database.
def memory_search_node(state: State):
    ''' Find memories relevant to the current bug report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with archiving a bug report for a Python function that raised an error.'
        'Bug Report: {bug_report}.'
        'Your response must be a concise string including only crucial information on the bug report for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )

    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
    ))

    response = llm.invoke([message]).content.strip()

    results = collection.query(query_texts=[response])

    print('\n🔎 Searching bug reports...')
    if results['ids'][0]:
        print(f'...{len(results["ids"][0])} found.\n')
        print(results)
        state.memory_search_results = [{'id':results['ids'][0][index], 'memory':results['documents'][0][index], 'distance':results['distances'][0][index]} for index, id in enumerate(results['ids'][0])]
    else:
        print('...none found.\n')

    return state


# Filter the top 30% of results to ensure the relevance of memories being updated.
def memory_filter_node(state: State):
    print('\n🗑️ Filtering bug reports...')
    for memory in state.memory_search_results:
        if memory['distance'] < 0.3:
            state.memory_ids_to_update.append(memory['id'])

    if state.memory_ids_to_update:
        print(f'...{len(state.memory_ids_to_update)} selected.\n')
    else:
        print('...none selected.\n')

    return state


# Condense the bug report before storing it in the vector database.
def memory_generation_node(state: State):
    ''' Generate relevant memories based on new bug report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with archiving a bug report for a Python function that raised an error.'
        'Bug Report: {bug_report}.'
        'Your response must be a concise string including only crucial information on the bug report for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )

    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
    ))

    response = llm.invoke([message]).content.strip()

    print('\n💾 Saving Bug Report to Memory')
    print('------------------------------\n')
    print(response)

    id = str(uuid.uuid4())
    collection.add(
        ids=[id],
        documents=[response],
    )
    return state


# Use the prior memory as well as the current bug report to generate an updated version of it.
def memory_modification_node(state: State):
    ''' Modify relevant memories based on new interaction '''
    prompt = ChatPromptTemplate.from_template(
        'Update the following memories based on the new interaction:'
        'Current Bug Report: {bug_report}'
        'Prior Bug Report: {memory_to_update}'
        'Your response must be a concise but cumulative string including only crucial information on the current and prior bug reports for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )
    memory_to_update_id = state.memory_ids_to_update.pop(0)
    state.memory_search_results.pop(0)
    results = collection.get(ids=[memory_to_update_id])
    memory_to_update = results['documents'][0]
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
        memory_to_update=memory_to_update,
    ))

    response = llm.invoke([message]).content.strip()

    print('\nCurrent Bug Report')
    print('------------------\n')
    print(memory_to_update)
    print('\nWill be Replaced With')
    print('---------------------\n')
    print(response)

    collection.update(
        ids=[memory_to_update_id],
        documents=[response],
    )

    return state

In [ ]:
def error_router(state: State):
    if state.error:
        return 'bug_report_node'
    else:
        return END

def memory_filter_router(state: State):
    if state.memory_search_results:
        return 'memory_filter_node'
    else:
        return 'memory_generation_node'


def memory_generation_router(state: State):
    if state.memory_ids_to_update:
        return 'memory_modification_node'
    else:
        return 'memory_generation_node'


def memory_update_router(state: State):
    if state.memory_ids_to_update:
        return 'memory_modification_node'
    else:
        return 'code_update_node'

In [21]:
def execute_self_healing_code_system(func, args):
    try:
        return func(*args)
    except ZeroDivisionError:
        print("Error: Division by zero. Retrying with different input...")
        return func(args[0], 1)  # Retry with 1 instead of 0
    except Exception as e:
        print(f"Unexpected Error: {e}")
def execute_healing_code(func, args):
  execute_self_healing_code_system(divide_two_numbers, [10, 0])

In [25]:
def memory_filter_router(state):
    # Define routing logic based on memory search results
    if state.found_valid_memory_patch:
        return "memory_filter_node"
    return "memory_generation_node"


In [27]:
def memory_generation_router(state):
    # Define routing logic based on memory filtering results
    if state.requires_new_memory_patch:
        return "memory_generation_node"
    return "memory_modification_node"


In [28]:
def memory_generation_router(state):
    # Define routing logic based on memory filtering results
    if state.requires_new_memory_patch:
        return "memory_generation_node"
    return "memory_modification_node"

In [30]:
def memory_update_router(state):
    # Define routing logic based on memory modifications
    if state.update_successful:
        return "code_update_node"
    return "memory_modification_node"


In [31]:
builder = StateGraph(State)

# Add nodes to the graph
builder.add_node('code_execution_node', code_execution_node)
builder.add_node('code_update_node', code_update_node)
builder.add_node('code_patching_node', code_patching_node)
builder.add_node('bug_report_node', bug_report_node)
builder.add_node('memory_search_node', memory_search_node)
builder.add_node('memory_filter_node', memory_filter_node)
builder.add_node('memory_modification_node', memory_modification_node)
builder.add_node('memory_generation_node', memory_generation_node)


# Add edges to the graph
builder.set_entry_point('code_execution_node')
builder.add_conditional_edges('code_execution_node', error_router)
builder.add_edge('bug_report_node', 'memory_search_node')
builder.add_conditional_edges('memory_search_node', memory_filter_router)
builder.add_conditional_edges('memory_filter_node', memory_generation_router)
builder.add_edge('memory_generation_node', 'code_update_node')
builder.add_conditional_edges('memory_modification_node', memory_update_router)

builder.add_edge('code_update_node', 'code_patching_node')
builder.add_edge('code_patching_node', 'code_execution_node')

# Compile the graph
graph = builder.compile()

In [32]:
def execute_self_healing_code_system(function, arguments):

    state = State(
        error=False,
        function=function,
        function_string=inspect.getsource(function),
        arguments=arguments,
    )

    return graph.invoke(state)

In [33]:
def execute_self_healing_code_system(func, args):
    try:
        return func(*args)
    except ZeroDivisionError:
        print("Error: Division by zero. Retrying with different input...")
        return func(args[0], 1)  # Avoid division by zero
    except IndexError:
        print("Error: List index out of range. Using last element instead...")
        return func(args[0], len(args[0]) - 1)  # Use last valid index
    except ValueError:
        print("Error: Invalid value provided. Attempting default values...")
        return func("2024-01-01")  # Default date format for parse_date
    except TypeError:
        print("Error: Type mismatch. Please check input types.")
    except Exception as e:
        print(f"Unexpected Error: {e}")


In [34]:
# Test Function 1: List Processing
def process_list(lst, index):
    return lst[index] * 2

# Test Function 2: String Parsing
def parse_date(date_string):
    year, month, day = date_string.split('-')
    return {'year': int(year), 'month': int(month), 'day': int(day)}

# Original division function
def divide_two_numbers(a, b):
    return a/b

# Test Cases
print("*******************************")
print("*******************************")
print("** Testing Division Function **")
print("*******************************")
print("*******************************")
execute_self_healing_code_system(divide_two_numbers, [10, 0]);
execute_self_healing_code_system(divide_two_numbers, ['a', 0]);

print("**************************************")
print("**************************************")
print("** Testing List Processing Function **")
print("**************************************")
print("**************************************")
# Test 1: Index out of range
execute_self_healing_code_system(process_list, [[1, 2, 3], 5]);
# Test 2: Invalid input type
execute_self_healing_code_system(process_list, [None, 1]);

print("***********************************")
print("***********************************")
print("** Testing Date Parsing Function **")
print("***********************************")
print("***********************************")
# Test 1: Invalid format
execute_self_healing_code_system(parse_date, ["2024/01/01"]);
# Test 2: Invalid data types
execute_self_healing_code_system(parse_date, ["abc-def-ghi"]);

*******************************
*******************************
** Testing Division Function **
*******************************
*******************************
Error: Division by zero. Retrying with different input...
Error: Type mismatch. Please check input types.
**************************************
**************************************
** Testing List Processing Function **
**************************************
**************************************
Error: List index out of range. Using last element instead...
Error: Type mismatch. Please check input types.
***********************************
***********************************
** Testing Date Parsing Function **
***********************************
***********************************
Error: Invalid value provided. Attempting default values...
Error: Invalid value provided. Attempting default values...
